In [1]:
%load_ext autoreload
%autoreload 2

# Simplified Self-attention

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [3]:
query = inputs[1] # the second input token embedding "journey"
attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
# The above are the un-normalized attention scores, we have to normalize them for a better representation of the attention scores

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print(f'Attention weights: {attn_weights_2_tmp}' )
print(f'Sum: {attn_weights_2_tmp.sum()}')

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: 1.0000001192092896


In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)
print(f'Attention weights: {attn_weights_2_naive}')
print(f'Sum: {attn_weights_2_naive.sum()}')

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(f"Attention weights: {attn_weights_2}")
print(f"Sum: {attn_weights_2.sum()}")

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: 1.0


In [8]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## Generalizing the 3 step process to get context vectors for all input embeddings

In [9]:
inputs.shape

torch.Size([6, 3])

In [10]:
attn_scores = torch.empty(inputs.size(0), inputs.size(0))

for i, x_i in enumerate(inputs):
    query = x_i
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_j, query) 

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [11]:
# The above is literally just matrix multiplication, we'll use matmul instead of slow for loops

attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [12]:
# we have the attention scores, now step 2 normalize them
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [13]:
# we have the normalized attn weights now step 3 sum them up to get the context vector for each input
context_vecs = attn_weights @ inputs
context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

# Implementing self-attention with trainable weights

In self-attention we introduce trainable attention weights. There are three trainable weight matrics: W_q, W_k, W_v. These are used to project the input tokens into query, key, and value vectors.

In [14]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [15]:
# Creating the weight matrics
torch.manual_seed(123)

w_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [16]:
query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value
print(query_2)

tensor([0.4306, 1.4551])


In [17]:
keys = inputs @ w_key
values = inputs @ w_value
print(f"keys.shape: {keys.shape}")
print(f"values.shape: {values.shape}")

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [18]:
# Computing the attention scores w_22
keys_2 = keys[1]
attn_score_22 = torch.dot(query_2, keys_2)
print(f"un-normalized attention score {attn_score_22}")

un-normalized attention score 1.8523844480514526


In [19]:
# Generalize to all attention scores via matrix multiplication
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [20]:
# Now we want the attention weights, we do softmax, but scale the attention scores by dividing them by the square root of the embedding dimension
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [21]:
# The last step is multiplying each value vector with its attention weight, then summing them to get the context vector (weighted sum)
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


## Generalization to compute all context vectors

### Now we will use the created self attention class v1 and v2

In [22]:
torch.manual_seed(123)
from self_attention import SelfAttention_v1, SelfAttention_v2

sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [23]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


### Excercise 3.1 Comparing self-attention v1 and self-attention v2

In [24]:
torch.manual_seed(123)
sa_v2 = SelfAttention_v2(d_in, d_out)
w_query = torch.nn.Parameter(sa_v2.w_query.weight.T)
w_key = torch.nn.Parameter(sa_v2.w_key.weight.T)
w_value = torch.nn.Parameter(sa_v2.w_value.weight.T)
print(w_query)
print(sa_v2.w_query.weight.T)
print(w_key)
print(sa_v2.w_key.weight.T)
print(w_value)
print(sa_v2.w_value.weight.T)

Parameter containing:
tensor([[-0.2354,  0.2177],
        [ 0.0191, -0.4919],
        [-0.2867,  0.4232]], requires_grad=True)
tensor([[-0.2354,  0.2177],
        [ 0.0191, -0.4919],
        [-0.2867,  0.4232]], grad_fn=<PermuteBackward0>)
Parameter containing:
tensor([[-0.4196,  0.2615],
        [-0.4590, -0.2133],
        [-0.3648,  0.2161]], requires_grad=True)
tensor([[-0.4196,  0.2615],
        [-0.4590, -0.2133],
        [-0.3648,  0.2161]], grad_fn=<PermuteBackward0>)
Parameter containing:
tensor([[-0.4900, -0.1135],
        [-0.3503, -0.4404],
        [-0.2120,  0.3780]], requires_grad=True)
tensor([[-0.4900, -0.1135],
        [-0.3503, -0.4404],
        [-0.2120,  0.3780]], grad_fn=<PermuteBackward0>)


In [25]:
# set the seed back to the previous one
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)

# Causal self-attention

### 4 step process: un-normalized attn scores -> normalized attn weights -> un-normalized masked attn scores -> normalized masked attn weights

In [26]:
# step 1 + 2
queries = sa_v2.w_query(inputs)
keys = sa_v2.w_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [27]:
# step 3
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [28]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [29]:
# step 4
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


### More efficient method of obtaining the masked attention weights
3 step process: un-normalized attention scores -> un-normalized mask attention scores -> normalized masked attention weights

In [30]:
# we can take advantage of softmax's exponential property by using -infinity above the diagonal instead of 0, since e^-infinity approaches 0
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [31]:
# Now we just apply softmax to these masked attention scores
attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### Dropout on attention weights

In [32]:
# attention weights before dropout
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

In [37]:
# now applying softmax
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


## Causal Attention class

In [44]:
from causal_attention import Causal_attention
torch.manual_seed(123)

batch = torch.stack((inputs, inputs), dim=0)

context_length = batch.shape[1]
ca = Causal_attention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print(context_vecs.shape)
print(context_vecs)

torch.Size([2, 6, 2])
tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)


# Multi-head attention

In [ ]:
from causal_attention import MultiHeadAttentionWrapper
torch.manual_seed(123)

context_length = batch.shape[1]
d_in, d_out = 3, 2

mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 4])


### Excercise 3.2 - Returning two-dimensional embedding vectors

In [ ]:
# output embedding 3rd dimension = d_out * num_heads, simply half the output dimension to get 
mha = MultiHeadAttentionWrapper(
    d_in, d_out=1, context_length=6, dropout=0.0, num_heads=2
)

context_vecs = mha(batch)
print(context_vecs)
print(context_vecs.shape)

tensor([[[-0.1530,  0.6575],
         [-0.2551,  0.6564],
         [-0.2885,  0.6537],
         [-0.2619,  0.5649],
         [-0.2745,  0.5175],
         [-0.2543,  0.4996]],

        [[-0.1530,  0.6575],
         [-0.2551,  0.6564],
         [-0.2885,  0.6537],
         [-0.2619,  0.5649],
         [-0.2745,  0.5175],
         [-0.2543,  0.4996]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 2])


### Testing Multi-head attention class

In [59]:
from multi_head_attention import MultiHeadAttention
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2

mha = MultiHeadAttention(
    d_in, d_out, context_length, num_heads=2, dropout=0.0
)

context_vecs = mha(batch)

print(context_vecs)
print(context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
torch.Size([2, 6, 2])


# Excercise 3.3 - GPT2 sized attention modules

In [64]:
# input and output dimensions are the same in gpt2 architecture
gpt2 = MultiHeadAttention(d_in=768, d_out=768, context_length=1024, num_heads=12, dropout=0.0)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

p_count = count_parameters(gpt2)
print(f"Number of parameters in the tiny gpt2 muti-head attention block: {p_count}")

Number of parameters in the tiny gpt2 muti-head attention block: 2360064
